1. Définir la question à répondre

1.1 Combien de municipalités sont à risque de feu de forêt au Québec ?
    1.1.1 Quelle est la zone de feu de forêt au Québec ?
    1.1.2 Combien il y a de municipalité inclus dans cette zone ?

1.2 Identifier les biens et les infrastructures les plus exposés aux feux de forêt à l’échelle des communautés.

2. Définir les sources de données pour cette analyse.
    2.1 Identifier la zone de couverture du jeu de données
    2.2 Identifier le CRS

In [36]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
import requests, zipfile, io

In [6]:
# 2. Définir les sources de données pour cette analyse

# Municipalités
regionalUrl = 'https://diffusion.mern.gouv.qc.ca/diffusion/RGQ/Vectoriel/Theme/Regional/SDA_100k/SHP/BDAT(adm)_SHP.zip'
# Aéroport
airportsUrl = 'https://ws.mapserver.transports.gouv.qc.ca/swtq?service=wfs&version=2.0.0&request=getfeature&typename=ms:evenements&outfile=Aeroport&srsname=EPSG:4326&outputformat=geojson'
# villages autochtones
indigenousVillagesUrl = 'https://diffusion.mern.gouv.qc.ca/Diffusion/RGQ/Documentation/BDVA/Index_BDVA.zip'

firesUrl = 'https://diffusion.mffp.gouv.qc.ca/Diffusion/DonneeGratuite/Foret/PERTURBATIONS_NATURELLES/Feux_foret/FEUX_PROV_GPKG.zip'

# Empreintes de bâtiments
buildingFootPrint = 'https://diffusion.mern.gouv.qc.ca/diffusion/RGQ/Vectoriel/Theme/Local/Batiments/FGDB/QC_600019-54_Prevention_risques_sinistres_2019_fgdb.zip'

In [7]:
# Downloads the data sources, then format to DataFrame

# Cas de la source de données à partir de geojson
responseBody = requests.get(airportsUrl, stream=True).json()
airports = gpd.GeoDataFrame.from_features(responseBody['features'], crs="EPSG:4326")
# airports.head(10)

# Loading all raw data can take a moment
rawDataRelativePath = './rawData'
# Use Case from .zip file
response = requests.get(firesUrl)
zipFile = zipfile.ZipFile(io.BytesIO(response.content))
zipFile.extractall(path=rawDataRelativePath)


In [32]:
targetFile = 'FEUX_PROV.gpkg'
pathAndFile = "%s/%s" % (rawDataRelativePath, targetFile)
provincialFiresGeom = gpd.read_file(pathAndFile)
# provincialFiresGeom.head()
provincialFiresGeom.crs

<Projected CRS: EPSG:32198>
Name: NAD83 / Quebec Lambert
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: Canada - Quebec.
- bounds: (-79.85, 44.99, -57.1, 62.62)
Coordinate Operation:
- name: Quebec Lambert Projection
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

# Loading all raw data can take a moment
rawDataRelativePath = './rawData'
# Use Case from .zip file
response = requests.get(regionalUrl)
zipFile = zipfile.ZipFile(io.BytesIO(response.content))
zipFile.extractall(path=rawDataRelativePath)

In [30]:
targetFile = 'regio_s.shp'
pathAndFile = "%s/Bdat/SHP/%s" % (rawDataRelativePath, targetFile)
regions = gpd.read_file(pathAndFile)
# regions.head()
# regions.explore()
regions.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [27]:
# regions.info()
# regions.describe()
regions.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [33]:
# Create a DataFrame that represent the entire province
quebecProvince = regions.dissolve(by='RES_DE_IND')
# quebecProvince.head()
# quebecProvince.explore()

In [48]:
# Create a custom shape
# source: https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html
rectangular = pd.DataFrame(
    {
        "Country": ["Canada"],
        "Coordinates": [
            "POLYGON ((-80 52, -55 52, -55 45, -80 45, -80 52))"
        ],
    }
)
rectangular["Coordinates"] = gpd.GeoSeries.from_wkt(rectangular["Coordinates"])
rectangular = gpd.GeoDataFrame(rectangular, geometry="Coordinates", crs="EPSG:4269")

# rectangular.crs
rectangular.explore()